# Install SAM

First we need to install SAM to install our first serverless APP

In [35]:
! curl -L https://github.com/aws/aws-sam-cli/releases/latest/download/aws-sam-cli-linux-x86_64.zip -o aws-sam-cli-linux-x86_64.zip
! unzip aws-sam-cli-linux-x86_64.zip -d sam-installation -qq
! ./sam-installation/install
! which sam
! sam --version

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 77.4M  100 77.4M    0     0  59.3M      0  0:00:01  0:00:01 --:--:--  120M
Archive:  aws-sam-cli-linux-x86_64.zip
caution: filename not matched:  -qq
Found preexisting AWS SAM CLI installation: /usr/local/aws-sam-cli/current. Please rerun install script with --update flag.
/usr/local/bin/sam
SAM CLI, version 1.126.0


## Let's download an application

We'll clone a sample application from git

The sample app takes an architecture diagram as a png image from you and transforms it into a blog post as a PDF using Amazon Bedrock

In [36]:
! git clone https://github.com/aws-samples/amazon-bedrock-claudev3-sonnet-blog-generation.git

fatal: destination path 'amazon-bedrock-claudev3-sonnet-blog-generation' already exists and is not an empty directory.


## Add a build step to the application

Go to folder src right click and choose 'create a new file' and create a new file named `requirements.txt`

Add the folloing line to the file and then press enter to add an empty new line

`fpdf2`

This is a library that is needed to create PDF files from the Lambda function

Now let's build our application

In [37]:
%%bash
cd amazon-bedrock-claudev3-sonnet-blog-generation
sam build

Building codeuri: /home/ec2-user/SageMaker/amazon-bedrock-claudev3-sonnet-blog-generation/src runtime: python3.12 architecture: x86_64 functions: InvokeBedrockClaudeV3Function
 Running PythonPipBuilder:ResolveDependencies
 Running PythonPipBuilder:CopySource



Build Succeeded

Built Artifacts  : .aws-sam/build
Built Template   : .aws-sam/build/template.yaml

Commands you can use next
[*] Validate SAM template: sam validate
[*] Invoke Function: sam local invoke
[*] Test Function in the Cloud: sam sync --stack-name {{stack-name}} --watch
[*] Deploy: sam deploy --guided


## Deploy the app with SAM

We need to interact with CLI and this will be hard from the notebook. Now click on the plus icon to open a new terminal

<img src="navi.png" alt="nav_bar" />

<img src="terminal.png" alt="terminal"/>


## Deploy the APP

execute
```bash
sam deploy -g
```

Add stack name as `bedrock`, region as `us-west-2` and press enter on the rest to use default inputs.

Once you get a confirmation message that app is deployed you can continue with testing

```bash
Successfully created/updated stack - bedrock in us-west-2
```

## Now let's try the app

Try to upload `sample-architecture.jpeg` image found in the repository `amazon-bedrock-claudev3-sonnet-blog-generation` to your input bucket. Then go and check the result in the output bucket
<img src="buckets.png" alt="nav_bar" />

## Let's add more custom observability

We can add more CloudWatch metrics to our function to indicate that certain parts of the code were invoked or to monitor some outputs of our processes.

Let's go the `template.yaml` file and add a policy to our Lambda

```yaml
- Version: '2012-10-17'
  Statement:
    - Effect: Allow
      Action: 
        - "cloudwatch:PutMetricData"
      Resource: "*"
```
Additionaly now we need to log something new from our function

Go inside the `src` folder and open `app.py`

Right after this line
```python
# Parse the response body
response_body = json.loads(response.get('body').read())
print(response_body)
```

We can add the following lines
```python
cloudwatch = boto3.client('cloudwatch')

cloudwatch.put_metric_data(
    MetricData = [
        {
            'MetricName': 'InputTokens',
            'Unit': 'Count',
            'Value': response_body["usage"]["input_tokens"]
        },
        {
            'MetricName': 'OutputTokens',
            'Unit': 'Count',
            'Value': response_body["usage"]["output_tokens"]
        },
    ],
    Namespace='MyAppTokens'
)
```

## Build, Deploy, Test

Now you need to prepat the following steps
- Build: we need to invoke `sam build` from the terminal to update the lambda code
- Deploy: We need to upload the changed lambda code and our changed template that now adds cloudwatch loging pemissions
- Finally we can reupload our sample image to input bucket, but now we need to observe additional metrics


## Check metrics in CLoudWatch metrics (takes 5 mins)

Now you can go check your new custom metric in the [Amazon Cloudwatch console](https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#metricsV2?graph=~(view~'timeSeries~stacked~false~metrics~(~(~'MyAppTokens~'InputTokens)~(~'.~'OutputTokens))~region~'us-west-2~start~'-PT1H~end~'P0D)&query=~'*7bMyAppTokens*7d)

## Final thoughts

You can also add custom metrics to your AWS StepFunctions workflows to keep AWS Lambda code clean

<img src="stepfunctions.png" alt="terminal"/>
